In [1]:
import gym
import random

In [2]:
env = gym.make('Taxi-v3').env
env.reset()


464

In [3]:
print(env.action_space)
print(env.observation_space)
len(env.P)


Discrete(6)
Discrete(500)


500

In [4]:
a = env.render(mode='ansi')
print(a)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [5]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table.shape
q_table
print('aaa')

aaa


In [6]:
%%time
from IPython.display import clear_output

alpha = 0.1
gamma = 0.6
epsilon = 0.1

for i in range(100000):
  estado = env.reset()

  penalidades, recompensa = 0, 0
  done = False
  while not done:
    # Exploração
    if random.uniform(0, 1) < epsilon:
      acao = env.action_space.sample()
    # Exploitation
    else:
      acao = np.argmax(q_table[estado])

    proximo_estado, recompensa, done, info = env.step(acao)

    q_antigo = q_table[estado, acao]
    proximo_maximo = np.max(q_table[proximo_estado])

    q_novo = (1 - alpha) * q_antigo + alpha * (recompensa + gamma * proximo_maximo)
    q_table[estado, acao] = q_novo

    if recompensa == -10:
      penalidades += 1

    estado = proximo_estado

  if i % 100 == 0:
    clear_output(wait=True)
    print('Episódio: ', i)

print('Treinamento concluído')



Episódio:  99900
Treinamento concluído
CPU times: total: 46.5 s
Wall time: 44.9 s


In [7]:
env.reset()
a = env.render(mode='ansi')
print(a)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [8]:
# 0 = south 1 = north 2 = east 3 = west 4 = pickup 5 = dropoff
env.encode(2,2,0,3)


243

In [9]:
q_table[243]

array([ -2.48235694,  -2.48236338,  -2.48236486,  -2.4510224 ,
       -11.46903275, -11.46771257])

In [10]:
total_penalidades = 0
episodios = 50
frames = []

for _ in range(episodios):
  estado = env.reset()
  penalidades, recompensa = 0, 0
  done = False
  while not done:
    acao = np.argmax(q_table[estado])
    estado, recompensa, done, info = env.step(acao)

    if recompensa == -10:
      penalidades += 1
    
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': estado,
        'action': acao,
        'reward': recompensa
    })

  total_penalidades += penalidades

print('Episódios', episodios)
print('Penalidades', total_penalidades)

Episódios 50
Penalidades 0


In [11]:
frames[0]

{'frame': '+---------+\n|\x1bR\x1b: | : :G|\n| : | : : |\n| : :\x1b \x1b: : |\n| | : | : |\n|\x1bY\x1b| : |B: |\n+---------+\n  (South)\n',
 'state': 242,
 'action': 0,
 'reward': -1}

In [12]:
from time import sleep
for frame in frames:
  clear_output(wait=True)
  print(frame['frame'])
  print('Estado', frame['state'])
  print('Ação', frame['action'])
  print('Recompensa', frame['reward'])
  sleep(.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Estado 16
Ação 1
Recompensa -1


KeyboardInterrupt: 